In [4]:
# establish libraries
import requests
import json
import pandas as pd
import gspread

In [ ]:
# Set your API key
users_api_key = input("enter the api key: ")

In [12]:
#function to use the facilities endpoint to find the campground ID by name
def get_campground_id(api_key, campground_name):
    base_url = "https://ridb.recreation.gov/api/v1"
    endpoint = "/facilities"
    url = base_url + endpoint

    headers = {
        "Accept": "application/json",
        "apikey": api_key
    }

    params = {
        "query": campground_name,
        "limit": 1
    }

    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()
        print(f"http response code: {response.status_code}")
        campgrounds_info = response.json()
        if "RECDATA" in campgrounds_info and len(campgrounds_info["RECDATA"]) > 0:
            campground_id = campgrounds_info["RECDATA"][0]["FacilityID"]
            return campground_id
        else:
            print("Campground not found.")
            return None
    except requests.exceptions.RequestException as e:
        print("Error occurred:", e)
        return None




In [8]:
#enter a campground name
campground_name = input("enter a campground name: ")

enter a campground name: Blackwoods Campground


In [13]:
# run the get_campground_id function to get the campground ID
campground_id_found = get_campground_id(users_api_key, campground_name)

http response code: 200


In [14]:
#conditional statement to print the campground id
if campground_id_found:
    print("Hey, you found the Campground ID:", campground_id_found)
else:
    print("Failed to pull the campground ID.")

Hey, you found the Campground ID: 232508


In [15]:
#function to use the facilities endpoint for details on campsites
def get_available_campsites(api_key, campground_id, start_date, end_date):
    base_url = "https://ridb.recreation.gov/api/v1"
    endpoint = f"/facilities/{campground_id}/campsites"
    url = base_url + endpoint

    headers = {
        "Accept": "application/json",
        "apikey": api_key
    }

    params = {
       
       "start_date": start_date,
        "end_date": end_date,
        "current": "true"
        #"limit": 10  # Change the limit as needed
    }

    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()
        print(f"http response code: {response.status_code}")
        campsites_info = response.json()
        return campsites_info
    except requests.exceptions.RequestException as e:
        print("Error occurred:", e)
        return None



In [16]:
#set date parameters
START_DATE = "2023-07-01"
END_DATE = "2023-07-07"

In [17]:
# use the function
campsites_info = get_available_campsites(users_api_key, campground_id_found, START_DATE, END_DATE)


http response code: 200


In [18]:
campsites_data = campsites_info["RECDATA"]

In [19]:
#convert results to a pandas dataframe
campsites_df = pd.DataFrame(campsites_data)

In [20]:
#clean up the data
c2 = campsites_df.drop(["ATTRIBUTES", "ENTITYMEDIA", "PERMITTEDEQUIPMENT"], axis=1)

Exporting to Google Sheets

In [ ]:
#authenticate to google drive
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)


In [ ]:
#create a google sheet
spreadsheet = gc.create('Acadia Campsite Data')

In [ ]:
#get the sheet
sheet = spreadsheet.get_worksheet(0)

In [ ]:
# Convert the DataFrame to a list
data_values = [c2.columns.tolist()] + c2.values.tolist()

In [ ]:
# fill google sheet with data
sheet.update(data_values)

In [ ]:
#list of spreadsheets in google drive
spreadsheet_list = gc.list_spreadsheet_files()


In [ ]:
print(spreadsheet_list)

In [ ]:
#use the spreadsheet ID
spreadsheet2 = gc.open_by_key("XXX")


In [ ]:
#add a new sheet
new_sheet = spreadsheet2.add_worksheet(title="Blackwoods", rows=100, cols=13)


In [ ]:
new_sheet.update(data_values)